In [18]:
# scraping libraries to read html code
import requests
from bs4 import BeautifulSoup
import json

In [19]:
# pandas library to create dataframe
# re for match
import pandas as pd
import numpy as np
import re

In [20]:
# standard library to count frequency 
import collections 

In [6]:
# Direct to initial results
url = 'https://www.yelp.com/search?find_desc='
keyword = input('What do you want to eat ? ')
address = input('Where are you ? ')

url = 'https://www.yelp.com/search?find_desc='+ keyword + '&find_loc=' + address 
url

What do you want to eat ? pizza
Where are you ? new york


'https://www.yelp.com/search?find_desc=pizza&find_loc=new york'

In [7]:
# Checking if the url is "valid", the status code should be 200
response = requests.get(url)
response.status_code

200

In [17]:
# extract the html code from the web page = https://www.yelp.com/search?find_desc=Restaurants&find_loc=New+York,+NY&start=30
results_page = BeautifulSoup(response.content,'lxml')

In [ ]:
# creating a list of businesses with the html code of the business as elements of the list
businesses = results_page.find_all('a',class_ = "biz-name js-analytics-click")

In [ ]:
# selecting the first element (a particular restaurant in our case) of the list businesses and looking in the
#ignore the first element of the businesses list, it is an ad by yelp, regular list start from the second (index = 1)
# html code for its name
business = businesses[1]
name = business.find('span').get_text()

# getting the link to the yelp page this particular restaurant
link = 'https://www.yelp.com' + business.get('href')

In [ ]:
print(name,link)

In [ ]:
# extracting the html code of the web page associated with the first business of our list businesses
response_business = requests.get(link)
results_business_page = BeautifulSoup(response_business.content,'lxml')

In [ ]:
# looking in the html page of the restaurant for latitude and longitude coordinates of the restaurant
# the json object json_map contains these coordinates
json_map = results_business_page.find('div', class_ = "lightbox-map hidden").get('data-map-state')

# we can convert this json object to a python dictionnary using the following method
dict_map = json.loads(json_map)

# the relevant keys in this dict are "markers", "location" and "latitude" and "longitude"
latitude = dict_map["markers"][1]["location"]["latitude"]
longitude = dict_map["markers"][1]["location"]["longitude"]

In [ ]:
json_map

In [ ]:
print(latitude,longitude)

In [ ]:
# The category can be found in the 'a' tag under "span", class_ = 'category-str-list' tag, 
category= results_business_page.find("span", class_ = 'category-str-list').find('a').get_text()

In [ ]:
category

In [ ]:
# the price range ('$','$$','$$$', or '$$$$') can be found in the "span", class_ = 'business-attribute price-range' tag
# the price range is not always available, that is why we need to catch errors 
# if necessary (hence the try: except) structure

try:
    price_range = results_business_page.find("span", class_ = 'business-attribute price-range').get_text()
except:
    price_range = None

In [ ]:
print(price_range)

In [ ]:
# the ratings associated to a particular restaurant are in the "div", itemprop = 'aggregateRating' tag
# the rating value (between 0 and 5) can be accessed through ratings (= results_business_page.find("div", 
# itemprop = 'aggregateRating') at the tag "meta" and the attribute 'content'
# the number of reviews is available through ratings at the 'span' tag

ratings = results_business_page.find("div", itemprop = 'aggregateRating')
rating_value = ratings.find("meta").get('content')
review_count = ratings.find("span").get_text()

In [ ]:
print(rating_value,review_count)

In [ ]:
# the phone number can be accessed at the tag 'span',itemprop = "telephone"
# phone numbers are not always available so we need to catch errors using the try:... except: structure

try:
    phone_number = results_business_page.find('span',itemprop = "telephone").get_text().strip()
except:
    phone_number = None   

In [ ]:
print(phone_number)

In [ ]:
# We want to know if the restaurant is currently open: this information is 
# given at the 'span', class_ = "nowrap extra open" tag. This information is not always 
# available that is why we use the try: ... except:... structure

try:
    open_ = results_business_page.find('span', class_ = "nowrap extra open").get_text()
except:
    open_ = None

In [ ]:
print(open_)

In [ ]:
try:
    addr = results_business_page.find('strong', class_ = "street-address").get_text().strip()
except:
    addr = None
addr

In [ ]:
try:
    web = results_business_page.find('span', class_ = "biz-website js-biz-website js-add-url-tagging").find('a').get_text()
except:
    web = None
web

In [4]:
#want to put all infomations we want to get from one business into one funtion
#input: specific restaurant url link
#output: all details we tested above
def get_info(link):
    response_business = requests.get(link)
    results_business_page = BeautifulSoup(response_business.content,'lxml')
    
    try:
        addr = results_business_page.find('strong', class_ = "street-address").get_text().strip()
    except:
        addr = None

    try:
        json_map = results_business_page.find('div', class_ = "lightbox-map hidden").get('data-map-state')
        dict_map = json.loads(json_map)
        lat = dict_map["markers"][1]["location"]["latitude"]
        lon = dict_map["markers"][1]["location"]["longitude"]
    except: 
        json_map = None 

    try: 
        category = results_business_page.find("span", class_ = 'category-str-list').find('a').get_text()
    except:
        category = None
        
    try:
        price_range = results_business_page.find("span", class_ = 'business-attribute price-range').get_text()
    except:
        price_range = None
        
    try: 
        rating_value = results_business_page.find("div", {'itemprop' : "aggregateRating"}).find("meta").get('content')
    except: 
        rating_value = None
          
    try:
        phone_number = results_business_page.find('span',itemprop = "telephone").get_text().strip()
    except:
        phone_number = None
        
    try:
        open_ = results_business_page.find('span', class_ = "nowrap extra open").get_text()
    except:
        open_ = 'closed now'
        
    try:
        web = results_business_page.find('span', class_ = "biz-website js-biz-website js-add-url-tagging").find('a').get_text()
    except:
        web = None
    
    try:
        moreinfo = results_business_page.find_all('dl')
        delivery = "No"
        for i in moreinfo[1:]:
            try:
                attr = i.find('dt',{'class':"attribute-key"}).get_text().strip()
                yon = i.find('dd').get_text().strip()
                if attr == 'Delivery':
                    delivery = yon
                    break
            except:
                delivery = delivery

    except:
        delivery = None
        
    return(addr,category,price_range,rating_value,phone_number,open_,web,delivery,lat,lon)
    

In [ ]:
#Get a list of url links of all businesses (or maybe the first 30)
#so that we could compile a list of details on, for example, the first 30 search-result restaurants
link_list = []

for i in range(1,len(businesses)):
    business = businesses[i]
    link = 'https://www.yelp.com' + business.get('href')
    link_list.append(link)

In [ ]:
#Create a list of restaurant names
name_list=[]
for i in range(1,len(businesses)):
    business = businesses[i]
    name = business.find('span').get_text()
    name_list.append(name)

In [ ]:
#Create a list of all restaurants' information
info_list=[]

for i in range(len(link_list)):
    info_list.append(get_info(link_list[i]))

In [5]:
def get_businesses():
    global keyword
    keyword = input('What do you want to eat ? ')
    global address
    address = input('Where are you ? ')
    limit = input('How many results do you want? ')
    filters = input('Do you want to sort by \n 1: price low to high  \n 2: price high to low  \n 3: ratings? \n Please enter the number 1/2/3:')
    
    #first page - figure out numbers of results per page
    url = 'https://www.yelp.com/search?find_desc='+ keyword + '&find_loc=' + address 
    response = requests.get(url)
    try:
        results_page = BeautifulSoup(response.content,'lxml')
    except:
        print("we didn't get back result page from yelp correctly")
    noperpage = results_page.find('span',{'class':'pagination-results-window'}).get_text().strip()[10:12]
    noperpage = int(noperpage)
    businesses = results_page.find_all('a',{'data-analytics-label':"biz-name"})[1:]
    global cuisine_styles
    cuisine_styles = results_page.find_all('span',class_="category-str-list")
    
    #start loop from the second page to get enough number of restaurants
    loop_range = (int(limit)-1)//noperpage
    for pagination in range(1, loop_range+1):
        url = 'https://www.yelp.com/search?find_desc='+ keyword + '&find_loc=' + address + '&start=' + str(pagination*noperpage)
        response = requests.get(url)
        try:
            results_page = BeautifulSoup(response.content,'lxml')
        except:
            print("we didn't get back result page from yelp correctly")
        
        businesses.extend(results_page.find_all('a',{'data-analytics-label':"biz-name"})[1:])
    businesses = businesses[:int(limit)]
    
    #generate details by get_info and format in dataframe
    business_list = []
    for business in businesses:
        name = business.find('span').get_text()
        link = 'https://www.yelp.com' + business.get('href')
        addr,category,price_range,rating_value,phone_number,open_,web,delivery,lat,lon= get_info(link)
        business_list.append([name,addr,open_,price_range,rating_value,category,phone_number,web,delivery,lat,lon])
    df = pd.DataFrame(business_list[:int(limit)+1], columns = ['name','address','open or closed','price range','rating value','category','phone','website','delivery service','lat','lon'])
    
    #sortby...
    pattern = r'rat'
    match = bool(re.search(pattern, filters))
    if match or filters == '3':
        df = df.sort_values(by='rating value',ascending=False)
    
    elif filters == 'price low to high' or filters == '1':
        df['length'] = df['price range'].apply(lambda x: len(x))
        df = df.sort_values(by='length',ascending=True)
        df = df.drop('length', axis=1)
        
    elif filters == 'price high to low' or filters == '2':
        df['length'] = df['price range'].apply(lambda x: len(x))
        df = df.sort_values(by='length',ascending=False)
        df = df.drop('length', axis=1)
    
    return(df)

In [6]:
res = get_businesses()

What do you want to eat ? lobster
Where are you ? boston
How many results do you want? 12
Do you want to sort by 
 1: price low to high  
 2: price high to low  
 3: ratings? 
 Please enter the number 1/2/3:2


In [7]:
# print(filters)
# print(re.search(pattern, filters))
res

,name,address,open or closed,price range,rating value,category,phone,website,delivery service,lat,lon
1,Neptune Oyster,"63 Salem StBoston, MA 02113",closed now,$$$,4.5,Seafood,(617) 742-3474,neptuneoyster.com,No,42.363271,-71.056058
5,Island Creek Oyster Bar,"500 Commonwealth AveBoston, MA 02215",closed now,$$$,4.5,Seafood,(617) 532-5300,islandcreekoysterbar.com,No,42.348682,-71.095118
7,Atlantic Fish,"761 Boylston StBoston, MA 02116",closed now,$$$,4.0,Seafood,(617) 267-4000,atlanticfishco.com,Yes,42.349222,-71.081127
11,Row 34,"383 Congress StBoston, MA 02210",closed now,$$$,4.0,American (New),(617) 553-5900,row34.com,No,42.349581,-71.047504
0,Luke’s Lobster Back Bay,"75 Exeter StreetBoston, MA 02116",closed now,$$,4.0,Seafood,(857) 350-4626,lukeslobster.com,Yes,42.348838,-71.079455
2,Yankee Lobster,"300 Northern AveBoston, MA 02210",closed now,$$,4.0,Seafood,(617) 345-9799,yankeelobstercompany.com,No,42.347922,-71.035988
3,James Hook & Co,"15-17 Northern AveBoston, MA 02110",closed now,$$,4.0,Seafood,(617) 423-5501,jameshooklobster.com,No,42.354962,-71.050974
4,Alive & Kicking Lobsters,"269 Putnam AveCambridge, MA 02139",closed now,$$,4.5,Seafood,(617) 876-0451,aliveandkickinglobsters.com,No,42.362676,-71.113820
6,Pauli’s,"65 Salem StBoston, MA 02113",closed now,$$,4.5,Sandwiches,(857) 284-7064,paulisnorthend.com,Yes,42.363244,-71.056085
8,North End Fish Market,"99 Salem StBoston, MA 02113",closed now,$$,4.5,Sushi Bars,(857) 362-7477,northendfish.com,Yes,42.363922,-71.055526


In [15]:
!pip install folium --upgrade

Requirement already up-to-date: folium in /Users/zijun/anaconda3/lib/python3.6/site-packages (0.7.0)
Requirement not upgraded as not directly required: branca>=0.3.0 in /Users/zijun/anaconda3/lib/python3.6/site-packages (from folium) (0.3.1)
Requirement not upgraded as not directly required: numpy in /Users/zijun/anaconda3/lib/python3.6/site-packages (from folium) (1.14.3)
Requirement not upgraded as not directly required: six in /Users/zijun/anaconda3/lib/python3.6/site-packages (from folium) (1.11.0)
Requirement not upgraded as not directly required: requests in /Users/zijun/anaconda3/lib/python3.6/site-packages (from folium) (2.18.4)
Requirement not upgraded as not directly required: jinja2 in /Users/zijun/anaconda3/lib/python3.6/site-packages (from folium) (2.10)
Requirement not upgraded as not directly required: chardet<3.1.0,>=3.0.2 in /Users/zijun/anaconda3/lib/python3.6/site-packages (from requests->folium) (3.0.4)
Requirement not upgraded as not directly required: idna<2.7,>=2

In [8]:
import folium
m = folium.Map(location=[40.75, -73.9625727],zoom_start=11.5)

In [9]:
# a marker is created for any restaurant: red for critical situation and orange for not critical
for k in range(len(res)):
    folium.Marker([res['lat'].iloc[k],res['lon'].iloc[k]],popup=res['name'].iloc[k]).add_to(m)
m

In [19]:
#Now, start to find the style of the search results so that we can make relevant recommendations later

cuisine_styles = results_page.find_all('span',class_="category-str-list")
#print(cuisine_styles)
# print(cuisine_styles[4].find('a').get_text())

styles = []
for i in range(len(cuisine_styles)):
    style = [x.get_text() for  x  in cuisine_styles[i].find_all('a')]
    styles.extend(style)

styles

['Pizza',
 'Italian',
 'Pizza',
 'Pizza',
 'Pizza',
 'Italian',
 'Pizza',
 'Pizza',
 'Italian',
 'Pizza',
 'Pizza',
 'Pizza',
 'Italian',
 'Pizza',
 'Pizza',
 'Italian',
 'Beer Bar',
 'Pizza',
 'Pizza',
 'Pizza',
 'Diners',
 'Pizza',
 'Italian',
 'Pizza',
 'Pizza',
 'Pizza',
 'Wine Bars',
 'Beer Bar',
 'Pizza',
 'Pizza',
 'Italian',
 'Pizza',
 'Italian',
 'Pizza',
 'Bars',
 'American (New)',
 'Pizza',
 'Pizza',
 'Italian',
 'Pizza',
 'Italian',
 'Wine Bars',
 'Pizza',
 'Bars',
 'Italian',
 'Pizza',
 'Pizza',
 'Vegan',
 'Italian',
 'Pizza',
 'Salad',
 'Italian',
 'Pizza',
 'Pizza']

In [47]:
# from several random keyword searching, we found that only the first two cuisine styles have significant counts
# e.g. out of the 30 results, the 3rd most common style appears around only 5 times
count = collections.Counter(styles).most_common(2)
print(count)

# Get a new keyword
if keyword == count[0][0].casefold():
    keyword2 = count[1][0].casefold()
else:
    keyword2 = count[0][0].casefold()

print(keyword2)

[('Pizza', 31), ('Italian', 13)]
italian


In [13]:
# Give users an opportunity to see recommendations relevant cuisine
# e.g. if originally searched for "pizza" but did not like the recommendations
# this function returns 10 more recommendations of "Italian" restaurants
def further_rec():
    reaction = input('Are you satisfied with your recommendations? (yes/no)')
    
    if reaction == 'yes':
        return 'Bon Appetit!'
    else:
        # Moved cuisine_styles to get_business() and make it global for access in this function
        styles = []
        for i in range(len(cuisine_styles)):
            style = [x.get_text() for  x  in cuisine_styles[i].find_all('a')]
            styles.extend(style)

        count = collections.Counter(styles).most_common(2)
        if keyword == count[0][0].casefold():
            keyword2 = count[1][0].casefold()
        else:
            keyword2 = count[0][0].casefold()

        url = 'https://www.yelp.com/search?find_desc='+ keyword2 + '&find_loc=' + address
        # new keyword but same address as from get_business()
        response = requests.get(url)
        try:
            results_page = BeautifulSoup(response.content,'lxml')
        except:
            print("we didn't get back result page from yelp correctly")
                
        businesses = results_page.find_all('a',{'data-analytics-label':"biz-name"})[1:]
        business_list = []
        for business in businesses:
            name = business.find('span').get_text()
            link = 'https://www.yelp.com' + business.get('href')
            addr,category,price_range,rating_value,phone_number,open_,web,delivery,lat,lon= get_info(link)
            business_list.append([name,open_,price_range,rating_value,category,phone_number,web,delivery,lat,lon])
                                 
        df = pd.DataFrame(business_list[:11], columns = ['name','open or closed','price range','rating value','category','phone','website','delivery service','lat','lon'])
        print('Take a look at the following 10 restaurants!')                                   
        return df

In [14]:
fr = further_rec()
fr

Are you satisfied with your recommendations? (yes/no)no
Take a look at the following 10 restaurants!


,name,open or closed,price range,rating value,category,phone,website,delivery service,lat,lon
0,Vanguard Wine Bar,closed now,$$,4.0,Wine Bars,(212) 799-9463,vanguard-nyc.com,No,40.776280,-73.983222
1,Vin Sur Vingt,closed now,$$,4.5,Wine Bars,(646) 895-9944,vsvwinebars.com,No,40.784938,-73.972925
2,Lilly’s Cocktail and Wine,Open now,$$,4.0,American (New),(212) 799-4140,lillys-cocktail-and-wine.busi…,No,40.777025,-73.978458
3,Amelie,Open now,$$$,4.5,French,(646) 422-7167,ameliewinebar.com,Yes,40.788773,-73.974670
4,Le Pif,Open now,$$,4.0,French,(212) 799-2253,lepifwinebars.com,No,40.777484,-73.981793
5,Pour,Open now,$$,4.0,"Beer, Wine & Spirits",(212) 501-7687,pourwines.com,No,40.780591,-73.979965
6,Vino Levantino,closed now,$$,4.0,Mediterranean,(212) 280-3333,vinolevantino.com,Yes,40.792948,-73.972078
7,Da Capo Columbus,Open now,$$,4.0,Italian,(646) 882-0197,dacaponyc.com,Yes,40.779635,-73.977485
8,The Milling Room,Open now,$$$,4.0,American (New),(212) 595-0380,themillingroom.com,No,40.783747,-73.974468
9,Bin 71,Open now,$$,3.5,Wine Bars,(212) 362-5446,bin71.com,No,40.776459,-73.979252


In [18]:
n = folium.Map(location=[40.8075355, -73.9625727],zoom_start=14)

In [19]:
for k in range(len(fr)):
    folium.Marker([fr['lat'].iloc[k],fr['lon'].iloc[k]],popup=fr['name'].iloc[k]).add_to(n)
n

In [ ]:
#workspace with saved results page
url = 'https://www.yelp.com/search?find_desc=lobster&find_loc=new+york&start=60'
response = requests.get(url)
try:
    results_page = BeautifulSoup(response.content,'lxml')
except:
    print("we didn't get back result page from yelp correctly")
  

In [ ]:
businesses = results_page.find_all('a',{'data-analytics-label':"biz-name"})[1:]
businesses.extend(businesses)
businesses

In [5]:
# Yelp is also providing API keys to do some web scraping on there website. We created our own API key to be able
# to do some web scraping when limited by the direct html scraping method we were using before

with open('./YelpAPIKeys.txt','r') as f:
    count = 0
    for line in f:
        if count == 0:
            CLIENT_ID = line.strip()
        if count == 1:
            API_KEY = line.strip()
        count+=1

In [6]:
print(CLIENT_ID,API_KEY)

lAFFi3W2fS4DEF5N0746Pg -lBWAWUBZqrndjWzjK9RrS3OSpCA_Haz_hjDutky0OjQcLXiLO7AAl4K_JlelcZSRNmysHWMfmHFLpf2QsPtWY1uCRIgSMJbdsqV6oTv8z00qR_lpJTkzzRa5zraW3Yx


In [29]:
CLIENT_ID = 'IUrGaOc4Tcb-CAL7_MZXsQ'
API_KEY = 'LLP0K4NgnF3ARH0O8x7pTHQ1-TpGU2aC94R1BIlWQpi5OFr6jwBj5-2U2wFJPnB_uoNtr4YXRMKEj4EXUp5MU2uXmihfv3hMX1WXE69POyI-gEHmcRYa6sSseV3jW3Yx'

In [30]:
API_HOST = 'https://api.yelp.com' # this is the API url header
SEARCH_PATH = '/v3/businesses/search' # this is the path for an API request to find businesses
BUSINESS_PATH = '/v3/businesses/'  # this is the path to get data for a single business

In [31]:
# this function creates a list of businesses, given an API key and a location like New York,...

def get_restaurants(api_key,location,keyword,number=50):
    import requests
    
    #First we get the access token
    #Set up the search data dictionary
    search_data = {
    'term': "restaurant",
    'location': location.replace(' ', '+'),
    'limit': number,
    'categories': keyword
    }
    url = API_HOST + SEARCH_PATH
    headers = {
        'Authorization': 'Bearer %s' % api_key,
    }
    response = requests.request('GET', url, headers=headers, params=search_data).json()
    businesses = response.get('businesses')
    return businesses

In [39]:
restaurants_data = get_restaurants(API_KEY,"New York",'pizza',number=50)

In [40]:
# This function creates a dataframe based on the data that we extracted from yelp website, 
# using the function get_restaurants

def data_output(restaurants_data):
    df = pd.DataFrame(columns = ['name','is_closed','latitude','longitude','price','rating','categories','phone'])
    for i in range(len(restaurants_data)):
        name = restaurants_data[i]['name']
        is_closed = restaurants_data[i]['is_closed']
        latitude = restaurants_data[i]['coordinates']['latitude']
        longitude = restaurants_data[i]['coordinates']['longitude']
        try:
            price = restaurants_data[i]['price']
        except:
            price = None
        rating = restaurants_data[i]['rating']
        categories = []
        for categorie in restaurants_data[i]['categories']:
            categories.append(categorie['title'])
        phone = restaurants_data[i]['phone']
        
        df.loc[i] = [name,is_closed,latitude,longitude,price,rating,categories,phone]
    df.set_index('name', inplace = True)
    return(df)

In [48]:
def web_scrapingYelp():
    import requests
    
    keyword = input('What do you want to eat ? ')
    address = input('Where are you ? ')
    limit = input('How many results do you want? ')

    search_data = {
    'term': "restaurant",
    'location': address.replace(' ', '+'),
    'limit': limit,
    'categories': keyword 
    }
    url = API_HOST + SEARCH_PATH
    headers = {
        'Authorization': 'Bearer %s' % API_KEY,
    }
    response = requests.request('GET', url, headers=headers, params=search_data).json()
    businesses = response.get('businesses')
    
    df = data_output(businesses)
    return df

In [49]:
web_scrapingYelp()


What do you want to eat ? burgers
Where are you ? New York
How many results do you want? 5


,is_closed,latitude,longitude,price,rating,categories,phone
name,,,,,,,
Burger & Lobster,False,40.740070,-73.993440,$$,4.0,"[Seafood, Burgers, American (New)]",+16468337532
Burger & Lobster,False,40.755960,-73.985320,$$,4.0,"[Seafood, Burgers]",+19175659044
Emily,False,40.683519,-73.966413,$$,4.0,"[Pizza, Burgers, American (Traditional)]",+13478449588
Lillie's Victorian Establishment,False,40.761463,-73.986052,$$,4.0,"[Bars, Gastropubs, Burgers]",+12129574530
Black Tap Craft Burger & Beers,False,40.763470,-73.979780,$$,4.0,"[Burgers, Beer, Wine & Spirits, Bars]",+12123154356
